In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from tavily import TavilyClient
import os
import sqlite3
from langchain_google_genai import ChatGoogleGenerativeAI

In [7]:
class AgentState(TypedDict):
    task: str # Human input - for which the essay is gonna be written.
    plan: str # Key to keep track of plan
    draft: str # Draft of essay
    critique: str # Key for critique
    content: List[str] # Key that keeps track of list of documents that tavily has researched & come back with
    # Gonna be used in criteria to decide, if to stop.
    revision_number: int # No. of revision we made.
    max_revisions: int # Max revisions we wanna make.

In [9]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [11]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [13]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [15]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

In [17]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [19]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

C:\Users\SamClitusFernando\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
tavily = TavilyClient()

In [23]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [25]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [27]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [29]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [31]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [33]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [35]:
builder = StateGraph(AgentState)

In [37]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [39]:
builder.set_entry_point("planner")

In [41]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    # output of should continue..
    {END: END, "reflect": "reflect"}
)

In [43]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [45]:
# Creating memory...
sqlite_conn = sqlite3.connect("checkpoints.sqlite", check_same_thread=False)
memory = SqliteSaver(sqlite_conn)

In [47]:
# Building graph..
graph = builder.compile(checkpointer=memory)

In [49]:
model = ChatGoogleGenerativeAI(model="gemini-1.5-pro", api_key=os.getenv("GOOGLE_API_KEY"))

In [52]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': "what is the difference between langchain and langsmith",
    "max_revisions": 2,
    "revision_number": 1``,
    "content": []
}, thread):
    print(s)

{'planner': {'plan': "## Essay Outline: LangChain vs. LangSmith: Two Sides of the LLM Application Coin\n\n**I. Introduction**\n\n*   **Hook:** Start with a compelling anecdote or observation about the growing complexity of LLM application development.\n*   **Context:** Briefly introduce Large Language Models (LLMs) and the challenges developers face in building robust and reliable applications around them.\n*   **Thesis Statement:**  LangChain and LangSmith represent distinct yet complementary approaches to LLM application development. LangChain provides the building blocks and framework, while LangSmith focuses on the monitoring, debugging, and iterative improvement process.  Understanding their differences is crucial for choosing the right tools for your project.\n\n**II. LangChain: The Building Blocks of LLM Applications**\n\n*   **What is LangChain?** Define LangChain as a framework specifically designed to simplify the development of applications powered by LLMs.\n*   **Key Featur

## Essay Writer Interface

In [57]:
import warnings
warnings.filterwarnings("ignore")

from helper import ewriter, writer_gui

ModuleNotFoundError: No module named 'helper'